In [5]:
#edited by Xinglu Cheng 2021.12.3
import ee
import os
from ipygee import Map 
from IPython.display import Image
import geemap

ee.Initialize()
gee_Map = geemap.Map(center=[35.0, 119.30], zoom=5)
gee_Map.add_basemap('Gaode.Normal')

#这里是调用ipygee的Map功能，也是很好的可视化工具，但是个人更喜欢吴秋生老师的geemap
# myMap = Map() 
# myMap.show()
# point = ee.Geometry.Point([119.30, 35.0])
# myMap.centerObject(point,zoom=5)
xiajiang_shp = r"G:\competition\xiajiang\峡江县.shp" #添加矢量数据
study_area = geemap.shp_to_ee(xiajiang_shp)    
# Map.centerObject(study_area, zoom=10)  #将矢量数据添加到底图中
roi = study_area.geometry()  #获取矢量的范围
#加入研究区
# study_area = ee.FeatureCollection("users/2210902126/Study_Area")
#myMap.setOptions("HYBRID")


#添加时间波段的函数
def createTimeBand(img): 
    date = img.metadata("date").subtract(1960)
    return date.rename("year").clip(study_area).addBands(img)

#遍历获取月降水数据，并循环计算每年的总降水量，存入list中
datalist=ee.List([])
year=1960
for i in range(60):
    datalist=datalist.add(ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filter(ee.Filter.date(ee.Date(str(year)+"-01-01"), ee.Date(str(year+1)+"-01-01"))).select('pr').sum().clip(study_area).set("date",ee.Number(year)))
    year += 1

varycollection=ee.ImageCollection.fromImages(datalist)#将list转为ImageCollection
varyCollection = varycollection.map(createTimeBand)#调用函数
linear = varyCollection.reduce(ee.Reducer.linearFit())#对每个像元线性回归
print(linear.getInfo())

#渲染显示
visualization = {
    'max':3,
    'min':-3,
    'palette': ['b30000','d7301f','ef6548','fc8d59','fdbb84','fdd49e','fee8c8','f7fcf0','e0f3db','ccebc5','a8ddb5','7bccc4','4eb3d3','2b8cbe','0868ac']
}
gee_Map.addLayer(linear.select('scale'),visualization,name='stable pre trend')#这里可以选择“offset”波段，渲染出来的则是各个像元的截距值
gee_Map


{'type': 'Image', 'bands': [{'id': 'scale', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'offset', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Map(center=[35.0, 119.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…